# Predicting gender based on features

In [16]:
import pandas as pd
import numpy as np
import cv2    
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
import glob
import os
from IPython.display import Image
import tensorflow as tf

import tensorflow.keras as keras
from keras.models import Sequential, Model 
from keras.layers import Dropout, Dense
from tensorflow.keras import Sequential
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [17]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Training the features to predict gender

In [18]:
main_folder = 'input/celeba-dataset/'
images_folder = main_folder + 'images/'
weights_folder = 'weights/'

TRAINING_SAMPLES = 10000
VALIDATION_SAMPLES = 2000
TEST_SAMPLES = 2000
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 16
NUM_EPOCHS = 20

## ATTRIBUTE DF

In [19]:
# # import the data set that include the attribute for each picture
# df_attr = pd.read_csv(main_folder + 'list_attr_celeba.csv')
# df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0

# df_attr['Male'] = df_attr['Male'].astype(str)
# df_attr['file_name']

In [20]:
# df_attr.set_index('file_name', inplace=True)
# df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
# df_attr.shape

## PARTITION DF (Important features)

In [21]:
# features = ['Goatee', 'Heavy_Makeup', 'Pale_Skin', 'Rosy_Cheeks', 'Wearing_Earrings', 'Wearing_Lipstick', 'Wearing_Necklace']

# df_partition = pd.read_csv(main_folder + 'list_eval_partition.csv')
# df_partition.head()
# df_partition['val'].value_counts().sort_index()

# # join the partition with the attributes
# df_partition.set_index('file_name', inplace=True)
# df_par_attr = df_partition.join(df_attr['Male'], how='inner')
# df_par_attr = df_par_attr.join(df_attr['Goatee'], how='inner')
# df_par_attr = df_par_attr.join(df_attr['Heavy_Makeup'], how='inner')
# df_par_attr = df_par_attr.join(df_attr['Pale_Skin'], how='inner')
# df_par_attr = df_par_attr.join(df_attr['Rosy_Cheeks'], how='inner')
# df_par_attr = df_par_attr.join(df_attr['Wearing_Earrings'], how='inner')
# df_par_attr = df_par_attr.join(df_attr['Wearing_Lipstick'], how='inner')
# df_par_attr = df_par_attr.join(df_attr['Wearing_Necklace'], how='inner')
# df_par_attr = df_par_attr.drop('val', axis=1)
# df_par_attr.head()

## Training features to give gender output

In [22]:
# def split_dataset(df, test_size, random_state=None):
#     label_encoder = preprocessing.LabelEncoder()

#     df['Male'] = label_encoder.fit_transform(df['Male'])

#     df_train, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
#     df_train2 = df_train.drop('Male', axis=1)
#     y_train2 = df_train['Male'].to_numpy()
#     df_test2 = df_test.drop('Male',axis=1)
#     y_test2 = df_test['Male'].to_numpy() 

#     return df_train2, y_train2, df_test2, y_test2

# def preprocess_dataset(df_train, df_test):

#     standard_scaler = preprocessing.StandardScaler()
#     df_train_scaled = standard_scaler.fit_transform(df_train)

#     df_test_scaled = standard_scaler.transform(df_test)

#     return df_train_scaled, df_test_scaled

# X_train, y_train, X_test, y_test = split_dataset(df_par_attr, test_size=0.3, random_state=0) # positive labels being encoded as 1

# X_train_scaled, X_test_scaled = preprocess_dataset(X_train, X_test)

In [23]:
# def FNN():
#     model = Sequential([
#         Dense(128, activation="relu", input_shape=(7,)),
#         Dropout(0.2),
#         Dense(128, activation="relu"),
#         Dropout(0.2),
#         Dense(128, activation="relu"),
#         Dropout(0.2),
#         Dense(1, activation="sigmoid")
#     ])

#     adamOptimizer = tf.keras.optimizers.Adam(0.001)

#     callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    
#     model.compile(optimizer=adamOptimizer,
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])

#     return model

# model = FNN()

# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# # history = model.fit(X_train_scaled, y_train, 
# #                     epochs=100, 
# #                     batch_size=256, 
# #                     verbose = 'auto', 
# #                     use_multiprocessing=True,
# #                     callbacks=[callback],
# #                     validation_data=(X_test_scaled, y_test))

In [24]:
# model.save('weights/celeba_fnn')

In [25]:
# model.evaluate(X_test_scaled, y_test,
#                batch_size=256, 
#                verbose = 'auto')

## Predicting gender based on features

In [26]:
model_1 = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/nn project/NN_PROJ-main/weights/celeba_all')
model_2 = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/nn project/NN_PROJ-main/weights/celeba_fnn')

FileNotFoundError: ignored

## Stage 1 - Image to 7 features

In [ ]:
test_folder = 'test/'
img_list = os.listdir(test_folder)
df_stage_1 = pd.DataFrame(columns=['Goatee', 'Heavy_Makeup', 'Pale_Skin', 'Rosy_Cheeks', 'Wearing_Earrings', 'Wearing_Lipstick', 'Wearing_Necklace'])


for img in img_list:
    pred = []
    image = tf.keras.preprocessing.image.load_img(os.path.join(test_folder, img))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  
    predictions = model_1.predict(input_arr)
    predictions = np.array(predictions)
    predictions = predictions.reshape(-1)
    for i in predictions:
        if i > 0.5:
            pred.append(1)
        else:
            pred.append(0)
    df_stage_1.loc[len(df_stage_1.index)] = pred
    
df_stage_1.insert(0, 'file_name', img_list)
df_stage_1

## Stage 2 - 7 features to gender

In [ ]:
df_stage_2 = df_stage_1.drop('file_name', axis=1)
df_stage_2 = df_stage_2.to_numpy()
df_stage_2 = np.asarray(df_stage_2).astype('float32')
pred_2 = model_2.predict(df_stage_2)
pred_2

In [ ]:
arr = []

pred_2 = pred_2.reshape(-1)
pred_2
for i in pred_2:
    if i > 0.5:
        arr.append(1)
    else:
        arr.append(0)
        
df_stage_1['Male'] = arr
true_gender = [1, 0, 0, 0, 1]
df_stage_1['True Male'] = true_gender
df_stage_1

## Feature importance (applicable to model 2)

In [ ]:
import shap
from sklearn import tree

df_features = df_stage_1.drop(['file_name', 'Male', 'True Male'], axis =1)
print(df_features)
# compute SHAP values
explainer = shap.DeepExplainer(model_2, X_train_scaled)
shap_values = explainer.shap_values(df_q4_scaled)

shap.initjs()
shap.force_plot(explainer.expected_value[0].numpy(), shap_values[0][0], features = df_q4_dropped.columns)